In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
import pickle
import json
path = "/home/ubuntu/mydata/pkl_data/distributable/level4"
dic = {}
with open(f"{path}/train.pkl", "rb") as f:
    dic["train"] = pickle.load(f)
with open(f"{path}/dev.pkl", "rb") as f:
    dic["dev"] = pickle.load(f)
with open(f"{path}/test.pkl", "rb") as f:
    dic["test"] = pickle.load(f)
src = pickle.load(open("/home/ubuntu/mydata/src_env.pkl", "rb"))
src = {x:y for x,y in src}

In [ ]:
def get_name(folder):
    prefix = "/home/ubuntu/mydata/virtual/env_"
    assert folder.startswith(prefix)
    return folder[len(prefix):].split("/")[0]

tot = {}
for k, lis in dic.items():
    for i, x in enumerate(lis):
        folder = x["folder"]
        pj = get_name(folder)
        assert pj in src
        assert src[pj] == folder
        if pj not in tot:
            tot[pj] = []
        item = {}
        item["name"] = x["name"]
        item["path"] = x["path"]
        item["position"] = x["position"]
        tot[pj].append(item)
print(len(tot))

In [ ]:
non_exist = []
for k in tot:
    try:
        with open(f'/home/ubuntu/mydata/signatureHelp/{k}_signature.json') as f:
            res = json.load(f)
    except Exception as e:
        print(k)
        with open(f'/home/ubuntu/mydata/signatureHelp/{k}.json', "w") as f:
            tmp = {}
            tmp["lis"] = tot[k]
            tmp["folder"] = src[k]
            tmp["extra"] = [f'/home/ubuntu/mydata/virtual/env_{k}/lib/python3.9/site-packages/']
            out = json.dumps(tmp, indent=2, separators=(",", ":"))
            f.write(out)
        non_exist.append(k)

    

In [ ]:
print(len(non_exist))
with open("/home/ubuntu/mydata/pkl_data/non_exist.pkl", "wb") as f:
    pickle.dump(non_exist, f)

In [ ]:
import os
s = 0
for k in tot:
    if os.path.exists(f'/home/ubuntu/mydata/signatureHelp/{k}_signature.json'):
        s+=1
    else:
        print(k, len(tot[k]))
print(s)

In [ ]:
# [Important]
# After you have finished `python main_signature.py`, you can begin to run this cell

overall = {}
for k, lis in dic.items():
    for i, x in enumerate(lis):
        overall[(x["name"], x["path"], x["position"]["line"], x["position"]["character"])] = (k,i)
for k in tot:
    try:
        with open(f'/home/ubuntu/mydata/signatureHelp/{k}_signature.json') as f:
            res = json.load(f)
    except:
        continue
    for x in res:
        key = (x["name"], x["path"], x["position"]["line"], x["position"]["character"])
        if key not in overall: continue
        a, b = overall[key]
        dic[a][b]["signature"] = x["signature"]

In [ ]:
for k, lis in dic.items():
    s = 0
    for i, x in enumerate(lis):
        if x.get("signature") is None:
            assert False

In [ ]:
print(dic["train"][100])

In [ ]:
for k, lis in dic.items():
    nw_lis = [x for x in lis if len(x["signature"])!=0]
    print(len(nw_lis), len(lis))
    with open(f"{path}_inter/{k}.pkl", "wb") as f:
        pickle.dump(nw_lis, f)